# 🔥 _DeepFire_: API Project for Fire Detection 🔥

In this project, you'll apply your skills at neural network development in a new way: taking a model that you've trained yourself and deploying it to a static webpage that you can work with to upload new images and get prediction accuracy results. 

This project will primarily focus on your abilities in creating and testing neural network architecture development. 

#### **Specifically, you'll be creating a convolutional neural network that can ingest Fire Detection Image Data and predict binary class values, similarly to what we've done with multilayer perceptrons in the past.**

Boilerplate and supporting architectures have been provided for a multitude of tasks ranging from data preprocessing, processing, ingestion, and predictive assessment – however, major tasks and design work will ultimately be left to you to approach and figure out ideal, optimized solutions. 

#### 🔹 General Importations

As always, we'll start with importing basic tools and functions for our task.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import utils

import os, PIL
from glob import glob

import tensorflow as tf

print(tf.__version__)

2.6.0


---

#### 🔎 Initializing Deep Learning Tools 🔍

---

> Your first task will be crucial to ensuring the successful implementation of the rest of your notebook. 
> 
> **Initialize each line with the correct function type from the TensorFlow documentation.**
> 
> Feel free to refer throughout the notebook and across previous notebooks to see which TensorFlow architectures you've used for similar tasks. 
> 
> To give you a guide for how this should look, you've been provided with a single correct function declaration in the form of `image_dataset_from_directory` at the end of the cell. 

---

In [2]:
#TODO's
""" Sequential Model Architecture """
Sequential = tf.keras.models.Sequential

""" Data Preprocessing Functions """
Resizing = tf.keras.layers.Resizing
Rescaling = tf.keras.layers.Rescaling

""" Data Augmentation Functions """
RandomFlip = tf.keras.layers.RandomFlip
RandomRotation = tf.keras.layers.RandomRotation
RandomZoom = tf.keras.layers.RandomZoom

""" Artificial Neural Network Layer Inventory """
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout

""" Convolutional Neural Network Layer Inventory """
Conv2D = tf.keras.layers.Conv2D
MaxPool2D = tf.keras.layers.MaxPool2D
Flatten = tf.keras.layers.Flatten

""" Residual Network Layer Inventory """
ResNet50 = tf.keras.applications.resnet50.ResNet50

""" Function to Load Images from Target Folder """
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory

#### 🔹 Precheck Image Dataset Sizes

If you've followed instructions carefully from the `project/PROJECT.md` instructions, the following dataset directory instantiations should work perfectly. 

If they do not, double-check to make sure you've saved your dataset to the appropriate location. 

In [3]:
# Use the `glob.glob` function to show how many images are in each folder
DATA_DIRECTORY = "./dataset/Images/"
FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Fire_Images/*"
NOT_FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Normal_Images/*"

print(f"Number of fire image samples: {len(glob(FIRE_IMAGES_PATTERN))}")
print(f"Number of non-fire image samples: {len(glob(NOT_FIRE_IMAGES_PATTERN))}")

Number of fire image samples: 541
Number of non-fire image samples: 541


#### 🔸 Load Dataset

Like we've done previously, let's set our batch size and image dimensions to work seamlessly with our configured model.

In [4]:
batch_size = 32
IMAGE_HEIGHT = IMAGE_WIDTH = 256

In [5]:
train = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 1082 files belonging to 2 classes.
Using 866 files for training.
2021-09-30 19:50:24.393642: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class_names = train.class_names

In [7]:
validation = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 1082 files belonging to 2 classes.
Using 216 files for validation.


From these results, we can actually see that we have a major class imbalance with our fire images representing our minority class. 

Let's go ahead and fix that by resampling our dataset. 

#### 🔹 Resample (Oversample) Minority Class

In [8]:
def resample_class_distribution(train, DATA_DIRECTORY=DATA_DIRECTORY, save=True):
    """ Helper function to resample class distribution for image dataset. """
    minority_class, majority_class = list(), list()
    for images, labels, in train.take(3):
        for image, label in zip(images, labels):
            if label == 0:
                minority_class.append(image.numpy().astype(np.uint8))
            else:
                majority_class.append(image.numpy().astype(np.uint8))
    FIRE_SIZE = len(glob(f"{DATA_DIRECTORY}/Fire_Images/*"))
    NOT_FIRE_SIZE = len(glob(f"{DATA_DIRECTORY}/Normal_Images/*"))
    upsampled_images = np.array(utils.resample(minority_class, replace=True, 
                                               n_samples=(NOT_FIRE_SIZE - FIRE_SIZE),
                                               random_state=42))
    if save == True:
        index = 0
        for image in upsampled_images:
            PATH = f"{DATA_DIRECTORY}/Fire_Images/new_fire_{index}.png"
            PIL.Image.fromarray(image).save(PATH)
            index += 1

In [9]:
resample_class_distribution(train)

2021-09-30 19:50:31.757755: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


You should now see that additional images have been generated to balance out both classes prior to predictive modeling.

**Go ahead and re-run the `Load Dataset` steps to see new generated dataset changes.**

#### 🔹 Pre-Optimize Image File Ingestion

This is an accessory step to optimize image data ingestion at the cost of slightly higher memory usage. No modifications are required for this function. 

In [10]:
def configure_performant_datasets(dataset, shuffling=None):
    """ 
    Custom function to prefetch and cache stored elements
    of retrieved image data to boost latency and performance
    at the cost of higher memory usage. 
    """
    AUTOTUNE = tf.data.AUTOTUNE
    # Cache and prefetch elements of input data for boosted performance
    if not shuffling:
        return dataset.cache().prefetch(buffer_size=AUTOTUNE)
    else:
        return dataset.cache().shuffle(shuffling).prefetch(buffer_size=AUTOTUNE)

In [11]:
train =         configure_performant_datasets(train, shuffling=1000)
validation =    configure_performant_datasets(validation)

---

#### 🔎 Create Resizing and Normalization Layers 🔍

---

> For this task, you'll declare your resizing and normalization layers using the layer architectures that you imported earlier. 
>
> Recall that for this step, we want to accomplish two key tasks: 
> - Resize all images to the predetermined square image dimensions as indicated by `IMAGE_HEIGHT` and `IMAGE_WIDTH`.
> - Scale all images so pixel values are within the range of (0., 1.) rather than the original (0., 255.).
>
> Additionally, since we're working with colorized image data, we'll want to ensure that our image rescaling/normalization step inputs images as stacks-of-three, since each image channel corresponds to red, green, and blue pixel values. 
>
> As always, refer to previous notebook documentation on image normalization for colorization if you need help.

---

In [12]:

resizing_layer = Resizing(IMAGE_HEIGHT, IMAGE_WIDTH)
normalization_layer = Rescaling(1./255, input_shape =(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

---

#### 🔎 Neural Network Architecture Creation 🔍

---

> Now time for the main event! 
> 
> Here, you'll be creating and instantiating your model architecture. 
> 
> For this assignment, you'll be creating a **Convolutional Neural Network** that can process Fire Detection images for predictive purposes.
> 
> _For this project, you will not be provided guidance as to how to design and implement your CNN architecture._
> 
> Refer to previous notebooks and challenges on CNNs as well as online documentation/resources for how to design CNN models on higher-order images. 
> 
> **This is a highly creative step, and there are no wrong answers; however, you will be assessed on your experimentation process and why you choose specific modeling layers, configurations, optimizers, regularizers, and overall design choices.**
>
> Light boilerplate will be provided to get you started, but as always, use any and all resources at your disposal to finish the job! 

---

In [13]:
#Creating the Layers Here as part of better format 

input_layer = tf.keras.layers.InputLayer(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))


conv_layer_1 = Conv2D(32, 
        kernel_size=(3,3),
        strides=(1,1),
        padding="same",
        activation="relu",
        input_shape=(32, 32,3)
        )

#Changes to CNN layer 
conv_layer_2 = Conv2D(64, 
        kernel_size=(3,3),
        strides=(1,1),
        padding='same',
        activation="relu"
        )

conv_layer_3 = Conv2D(128, 
        kernel_size=(3,3),
        strides=(1,1),
        padding='same',
        activation="relu"
        )

conv_layer_4 = Conv2D(192, 
        kernel_size=(3,3),
        strides=(1,1),
        padding='same',
        activation="relu"
        )

conv_layer_5 = Conv2D(256, 
        kernel_size=(3,3),
        strides=(1,1),
        padding='same',
        activation="relu"
        )

pooling_layer_1 = MaxPool2D(pool_size=(2,2), strides=(1,1), padding='same') 
pooling_layer_2 = MaxPool2D(pool_size=(2,2), padding='same')
pooling_layer_3 = MaxPool2D(pool_size=(2,2))
pooling_layer_4 = MaxPool2D(pool_size=(2,2))

dropout_layer_1 = Dropout(0.25)
dropout_layer_2 = Dropout(0.25)
dropout_layer_3 = Dropout(0.4)
dropout_layer_4 = Dropout(0.5)
dropout_layer_5 = Dropout(0.6)
dropout_layer_6 = Dropout(0.7)

dense_layer_1 = Dense(256, activation="relu")
dense_layer_2 = Dense(128, activation="relu")
dense_layer_3 = Dense(64, activation="relu")

flattening_layer = Flatten()

output_layer = Dense(1, activation="sigmoid")


#Resizing
resizing_layer = Resizing(IMAGE_HEIGHT, IMAGE_WIDTH)

#Rescaling
rescaling_layer = Rescaling(1./255, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

#RandomFlip
flipping_layer = RandomFlip("horizontal", input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

#RandomRotation
random_rotating_layer = RandomRotation(0.2)

#RandomZoom
random_zoom_layer = RandomZoom(0.2)






In [14]:
""" Sequential Model Architecture Setup """
model = Sequential()


""" CNN Layering Steps """
model.add(input_layer)

#Data Augmentation
model.add(resizing_layer)
model.add(normalization_layer)

model.add(conv_layer_1)
model.add(pooling_layer_1)
model.add(dropout_layer_1)


model.add(conv_layer_2)
model.add(pooling_layer_2)
model.add(dropout_layer_2)

model.add(conv_layer_3)
model.add(pooling_layer_3)
model.add(dropout_layer_3)

model.add(conv_layer_4)
model.add(pooling_layer_4)
model.add(dropout_layer_4)

model.add(flattening_layer)

model.add(dense_layer_1)
model.add(dropout_layer_5)
model.add(dense_layer_2)
model.add(dropout_layer_6)
model.add(dense_layer_3)
model.add(output_layer)


""" CNN Architecture Summarization """
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_1 (Resizing)        (None, 256, 256, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 256, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0

---

#### 🔎 Neural Network Configuration 🔍

---

> For this task, you'll compile your CNN architecture with appropriate parameters for loss calculation, optimization, and accuracy metrics.
> 
> As always, refer to previous notebooks, tutorials, and documentation for best-case parameters to use for image recognition models.

---

In [15]:
""" CNN Model Compilation """
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='nadam',
    metrics=['accuracy']
)

#Second change - adam to Nadam

---

#### 🔎 CNN Model Predictive Fitness 🔍

---

> For this task, you'll be taking your compiled model and fitting it against your training and validation data.
> 
> Keep in mind that there are several opportunities for further optimizing your workflow with techniques such as batch normalization, generator-based data feeding, etc. 
> 
> As always, refer to previous notebooks, tutorials, and documentation for designing model fitness with validation data. 

---

In [16]:
""" CNN Model Fitness and History Extraction """
#Third Change - number of epochs
epochs = 9
history = model.fit(train, validation_data = validation,
                    epochs=epochs)

Epoch 1/9
28/28 [==============================] - 155s 5s/step - loss: 0.8725 - accuracy: 0.5785 - val_loss: 0.6315 - val_accuracy: 0.7593
Epoch 2/9
28/28 [==============================] - 150s 5s/step - loss: 0.5474 - accuracy: 0.7575 - val_loss: 0.6231 - val_accuracy: 0.5324
Epoch 3/9
28/28 [==============================] - 147s 5s/step - loss: 0.5002 - accuracy: 0.8025 - val_loss: 0.3325 - val_accuracy: 0.8472
Epoch 4/9
28/28 [==============================] - 150s 5s/step - loss: 0.2815 - accuracy: 0.9088 - val_loss: 0.3322 - val_accuracy: 0.8611
Epoch 5/9
28/28 [==============================] - 146s 5s/step - loss: 0.2246 - accuracy: 0.9273 - val_loss: 0.1761 - val_accuracy: 0.9120
Epoch 6/9
28/28 [==============================] - 147s 5s/step - loss: 0.1706 - accuracy: 0.9480 - val_loss: 0.1596 - val_accuracy: 0.9213
Epoch 7/9
28/28 [==============================] - 147s 5s/step - loss: 0.1410 - accuracy: 0.9527 - val_loss: 0.1720 - val_accuracy: 0.9352
Epoch 8/9
28/28 [===

---

#### 🔎 CNN Model Evaluation 🔍

---

> For this task, you'll evaluate your CNN model using the validation dataset in order to calculate overall validation accuracy and loss.
> 
> As always, refer to previous notebooks, tutorials, and documentation for using the proper evaluation function for model prediction. 

---

In [17]:
""" CNN Model Predictive Evaluation """
model.evaluate(validation)

7/7 [==============================] - 6s 869ms/step - loss: 0.1355 - accuracy: 0.9491


[0.13546763360500336, 0.9490740895271301]

#### 🔹 Model State Saving

When you are satisfied with your model state configuration and performance and are ready to export the model's weights and parameters for deployment purposes, simply run the following function! 

In [18]:
def save_model(model, file_name, save_format):
    """ 
    Save the model weights and architecture.
    
    Parameters: 
       model(Model): keras Model object being saved
       file_name(str): name of the Hadoop file where
                       the whole model will be saved
       save_format(str): Indicates whether to save the model to the default
                         SavedModel('tf'), or HDF5('h5'), or 
                         use both H5 and JSON ('composite') formats. 
       Returns: None
    """
    MODEL_DIRECTORY = "../model"
    def __save_as_composite():
      """ Saving the model as H5 (for params) + JSON (for the architecture) """
      # Save the weights
      model.save_weights(f'{MODEL_DIRECTORY}/{file_name}_params.h5')
      # Save the architecture
      with open(f'{MODEL_DIRECTORY}/{file_name}_layers.json', 'w') as f:
          f.write(model.to_json())
    
    def __save_as_h5():
      """ Option 2: Saving whole model as a single H5 file (more storage) """
      model.save(f"{MODEL_DIRECTORY}/{file_name}.h5", save_format=save_format)

    # Call the appropiate save func
    if save_format == 'h5':
      __save_as_h5()
    elif save_format == 'composite':
      __save_as_composite()
    else:  # save as a SavedModel
      model.save(file_name)

In [19]:
save_model(model, "fire_cnn_classifier", "composite")

Now go ahead and complete the remaining tasks in `project/PROJECT.md` to complete this project successfully! 

---
---
---